In [11]:
import numpy as np
import torch
import math
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import optim
import os

In [12]:
PATH_base = './VGG-CIFAR10-const-umut'

In [13]:
'''VGG11/13/16/19 in Pytorch.'''
import torch
import torch.nn as nn


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


def test():
    net = VGG('VGG11')
    x = torch.randn(2,3,32,32)
    y = net(x)
    print(y.size())

# test()

In [14]:
batch_size= 10
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

lr_list = [0.005, 0.01, 0.015, 0.02]
# lr_list = [0.02]
criterion = nn.CrossEntropyLoss()
nodes = 1

Files already downloaded and verified
Files already downloaded and verified


In [15]:
# const lr
PATH_base = './VGG-CIFAR10-const-umut'

lr_list = [0.005, 0.01, 0.015, 0.02]
train_accuracy = []
train_error = []
for node in range(nodes):
    tmp_err = []
    tmp_acc = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model100' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in train_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_err.append(error/50000)
        tmp_acc.append(test_acc/50000)
        print(i)
    train_error.append(tmp_err)
    train_accuracy.append(tmp_acc)

test_accuracy = []
test_error = []
for node in range(nodes):
    tmp_acc = []
    tmp_err = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model100' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in test_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_acc.append(test_acc/10000)
        tmp_err.append(error/10000)
        print(i)
    test_error.append(tmp_err)
    test_accuracy.append(tmp_acc)

0
0
1
2
3
0
0
1
2
3


In [16]:
np.save('vgg-const-train-acc', train_accuracy)
np.save('vgg-const-test-acc', test_accuracy)
np.save('vgg-const-train-err', train_error)
np.save('vgg-const-test-err', test_error)

In [17]:
# cyclic lr
PATH_base = './VGG-CIFAR10-cyclic-umut'

lr_list = [0.005, 0.01, 0.015, 0.02]
train_accuracy = []
train_error = []
for node in range(nodes):
    tmp_err = []
    tmp_acc = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model100' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in train_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_err.append(error/50000)
        tmp_acc.append(test_acc/50000)
        print(i)
    train_error.append(tmp_err)
    train_accuracy.append(tmp_acc)

test_accuracy = []
test_error = []
for node in range(nodes):
    tmp_acc = []
    tmp_err = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model100' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in test_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_acc.append(test_acc/10000)
        tmp_err.append(error/10000)
        print(i)
    test_error.append(tmp_err)
    test_accuracy.append(tmp_acc)

0
0
1
2
3
0
0
1
2
3


In [18]:
np.save('vgg-cyclic-train-acc', train_accuracy)
np.save('vgg-cyclic-test-acc', test_accuracy)
np.save('vgg-cyclic-train-err', train_error)
np.save('vgg-cyclic-test-err', test_error)

In [19]:
# uniform lr
PATH_base = './VGG-CIFAR10-unif-umut'

lr_list = [0.005, 0.01, 0.015, 0.02]
train_accuracy = []
train_error = []
for node in range(nodes):
    tmp_err = []
    tmp_acc = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model100' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in train_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_err.append(error/50000)
        tmp_acc.append(test_acc/50000)
        print(i)
    train_error.append(tmp_err)
    train_accuracy.append(tmp_acc)

test_accuracy = []
test_error = []
for node in range(nodes):
    tmp_acc = []
    tmp_err = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model100' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in test_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_acc.append(test_acc/10000)
        tmp_err.append(error/10000)
        print(i)
    test_error.append(tmp_err)
    test_accuracy.append(tmp_acc)

0
0
1
2
3
0
0
1
2
3


In [20]:
np.save('vgg-unif-train-acc', train_accuracy)
np.save('vgg-unif-test-acc', test_accuracy)
np.save('vgg-unif-train-err', train_error)
np.save('vgg-unif-test-err', test_error)

In [21]:
# mc lr
PATH_base = './VGG-CIFAR10-mc-umut'

lr_list = [0.005, 0.01, 0.015, 0.02]
train_accuracy = []
train_error = []
for node in range(nodes):
    tmp_err = []
    tmp_acc = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model100' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in train_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_err.append(error/50000)
        tmp_acc.append(test_acc/50000)
        print(i)
    train_error.append(tmp_err)
    train_accuracy.append(tmp_acc)

test_accuracy = []
test_error = []
for node in range(nodes):
    tmp_acc = []
    tmp_err = []
    print(node)
    for i in range(len(lr_list)):
        PATH = PATH_base + '/LR' + '{}'.format(i)
        # tmp_path_mod = PATH  + '/node' +'{}'.format(node)+ '/model-1' +'.pth'
        tmp_path_mod = PATH  + '/model100' +'.pth'
        # tmp_path_mod = PATH + '/model900' +'.pth'
        tmp_net = torch.load(tmp_path_mod,map_location='cpu')
        test_acc = 0
        error = 0
        for data in test_loader:
            img, label = data
            # img=img.view(img.size(0),-1)
            img = Variable(img)
            label = Variable(label)
            out = tmp_net(img)
            loss = criterion(out, label)
            error += loss.data.item()
            _, pred = torch.max(out.data, 1)
            test_acc += pred.eq(label.view_as(pred)).sum().item()
        tmp_acc.append(test_acc/10000)
        tmp_err.append(error/10000)
        print(i)
    test_error.append(tmp_err)
    test_accuracy.append(tmp_acc)

0
0
1
2
3
0
0
1
2
3


In [22]:
np.save('vgg-mc-train-acc', train_accuracy)
np.save('vgg-mc-test-acc', test_accuracy)
np.save('vgg-mc-train-err', train_error)
np.save('vgg-mc-test-err', test_error)

In [28]:
test_error

[[0.010773289799690246,
  0.011110515737533569,
  0.011131219869852067,
  0.011102753084897995,
  0.011646000272035599,
  0.0118322182148695,
  0.011595230954885483,
  0.012870685708522797]]

In [29]:
test_accuracy

[[0.8414, 0.8452, 0.8498, 0.8482, 0.8461, 0.8505, 0.8513, 0.8457]]

In [25]:
train_acc = np.load("vgg-mc-train-acc.npy")
train_acc

array([[0.99996, 1.     , 0.99878, 0.99422]])